In [1]:
import types

In [2]:
async def async_function():
    return 1

In [3]:
async def async_generator():
    yield 1

In [4]:
type(async_function())

d:\python3.78\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: coroutine 'async_function' was never awaited
  """Entry point for launching an IPython kernel.


coroutine

In [5]:
type(async_generator())

async_generator

In [8]:
async_function().send(None)

StopIteration: 1

In [12]:
import os
import time
import sys

import requests

POP20_CC = ('CN IN US ID BR P1K NG BD RU JP MX PH VN ET EG DE IR TR CD FR').split()

BASE_URL = 'http://flupy.org/data/flags'

DEST_DIR = './downloads/'

def save_flag(img, filename):
    path = os.path.join(DEST_DIR, filename)
    with open(path, 'wb') as fp:
        fp.write(img)
        
def get_flag(cc):
    url = '{}/{cc}/{cc}.gif'.format(BASE_URL, cc=cc.lower())
    resp = requests.get(url)
    return resp.content

def show(text):
    print(text, end=' ')
    sys.stdout.flush()
    
def download_many(cc_list):
    for cc in sorted(cc_list):
        image = get_flag(cc)
        show(cc)
        save_flag(image, cc.lower() + '.gif')
        
    return len(cc_list)

def main(download_many):
    t0 = time.time()
    count = download_many(POP20_CC)
    elapsed = time.time() - t0
    msg = '\n{} flags downloaded in {:.2f}s'
    print(msg.format(count, elapsed))
    
if __name__ == '__main__':
    main(download_many)

BD BR CD CN DE EG ET FR ID IN IR JP MX NG P1K PH RU TR US VN 
20 flags downloaded in 21.82s


In [18]:
from concurrent import futures

from flags import save_flag, get_flag, show, main

MAX_WORKERS = 20

def download_one(cc):
    image = get_flag(cc)
    show(cc)
    save_flag(image, cc.lower() + '.gif')
    return cc

def download_many(cc_list):
    # 设定工作的线程数：允许的最大值与要处理的数量之间的 最小值，以免创建多余的线程
    workers = min(MAX_WORKERS, len(cc_list))
    with futures.ThreadPoolExecutor(workers) as executor:
        res = executor.map(download_one, sorted(cc_list))
        
    return len(list(res))

if __name__ == '__main__':
    main(download_many)

ConnectionError: HTTPConnectionPool(host='flupy.org', port=80): Max retries exceeded with url: /data/flags/bd/bd.gif (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001B7338DE608>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

In [19]:
def download_many(cc_list):
    cc_list = cc_list[:5]
    with futures.ThreadPoolExecutor(max_workers=3) as executor:
        to_do = []
        for cc in sorted(cc_list):
            future = executor.submit(download_one, cc)
            to_do.append(future)
            msg = 'Scheduled for {}: {}'
            print(msg.format(cc, future))
            
        results = []
        for future in futures.as_completed(to_do):
            res = future.result()
            msg = '{} result: {!r}'
            print(msg.format(future, res))
            results.append(res)
            
    return len(results)

In [4]:
from time import sleep, strftime
from concurrent import futures

def display(*args):
    print(strftime('[%H:%M:%S]'), end=' ')
    print(*args)
    
def loiter(n):
    msg = '{}loiter({}): doing nothing for {}s...'
    display(msg.format('\t'*n, n, n))
    sleep(n)
    msg = '{}loiter({}): done.'
    display(msg.format('\t'*n, n))
    return n * 10

def main():
    display('Script starting.')
    executor = futures.ThreadPoolExecutor(max_workers=3)
    results = executor.map(loiter, range(5))
    display('results:', results)
    display('Waiting for individual results:')
    for i, result in enumerate(results):
        display('result {}: {}'.format(i, result))
        
main()

[09:07:40] Script starting.
[09:07:40] loiter(0): doing nothing for 0s...
[09:07:40] loiter(0): done.
[09:07:40] 	loiter(1): doing nothing for 1s...
[09:07:40] 		loiter(2): doing nothing for 2s...
[09:07:40] [09:07:40] 			loiter(3): doing nothing for 3s...
results: <generator object Executor.map.<locals>.result_iterator at 0x00000218395B5F48>
[09:07:40] Waiting for individual results:
[09:07:40] result 0: 0
[09:07:41] 	loiter(1): done.
[09:07:41][09:07:41]  result 1: 10
				loiter(4): doing nothing for 4s...
[09:07:42] 		loiter(2): done.
[09:07:42] result 2: 20
[09:07:43] 			loiter(3): done.
[09:07:43] result 3: 30
[09:07:45] 				loiter(4): done.
[09:07:45] result 4: 40


In [6]:
import time
from tqdm import tqdm
for i in tqdm(range(1000)):
    time.sleep(.01)

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:10<00:00, 94.19it/s]


In [1]:
def get_flag(base_url, cc):
    url = '{}/{cc}/{cc}.gif'.format(base_url, cc=cc.lower())
    resp = requests.get(url)
    if resp.status_code != 200:
        resp.raise_for_status()
    return resp.content

def download_one(cc, base_url, verbose=False):
    try:
        image = get_flag(base_url, cc)
    except requests.exceptions.HTTPError as exc:
        res = exc.respone
        if res.status_code == 404:
            status = HTTPStatus.not_found
            msg = 'not found'
        else:
            raise
    else:
        save_flag(image, cc.lower90 + '.gif')
        status = HTTPStatus.ok
        msg = 'ok'
    if verbose:
        print(cc, msg)
        
    return Result(status, cc)

flags2_sequential.py：实现依序下载的 download_many
函数

In [2]:
def download_many(cc_list, base_url, verbose, max_req):
    counter = collections.Counter() #1
    cc_iter = sorted(cc_list) #2
    if not verbose:
        cc_iter = tqdm.tqdm(cc_iter) # 3
    for cc in cc_iter: #4
        try:
            res = download_one(cc, base_url, verbose) # 5
        except requests.exceptions.HTTPError as exc: #6
            error_msg = 'HTTP error {res.status_code} - {res.reason}'
            error_msg = error_msg.format(res=exc.response)
        except requests.exceptions.ConnectionError as exc: # 7
            error_msg = 'Connection error'
        else: #8
            error_msg = ''
            status = res.status
            
        if error_msg:
            status = HTTPStatus.error #9
        counter[status] += 1 #10
        if verbose and error_msg: #11
            print('*** Error for {}: {}.'.format(cc, error_msg))
            
    return counter #12

flags2_threadpool.py：完整的代码清单

In [ ]:
import collections
from concurrent import futures

import requests
import tqdm #1

from flags2_common import main, HTTPStatus #2
from flags2_sequential import download_one #3

DEFAULT_CONCUR_REQ = 30 #4
MAX_CONCUR_REQ = 1000 #5

def download_many(cc_list, base_url, verbose, concur_req):
    counter = collections.Counter()
    with futures.ThreadPoolExecutor(max_workers=concur_req) as executor: # 6
        to_do_map = {} #7
        for cc in sorted(cc_list): #8
            future = executor.submit(download_one, cc, base_url, verbose) #9
            to_do_map[future] == cc #10
        done_iter = futures.as_completed(to_do_map) #11
        if not verbose:
            done_iter = tqdm.tqdm(done_iter, total=len(cc_list)) # 12
        for future in done_iter: #13
            try:
                res = future.result() #14
            except requests.exceptions.HTTPError as exc: #15
                error_msg = 'HTTP {res.status_code} - {res.reason}'
                error_msg = error_msg.format(res=exc.response)
            except requests.exceptions.ConnectionError as exc:
                error_msg = 'Connection error'
            else:
                error_msg = ''
                status = res.status
            
            if error_msg:
                status = HTTPStatus.error
            counter[status] += 1
            if verbose and error_msg:
                cc = to_do_map[future] #16
                print('*** Error for {}: {}'.format(cc, error_msg))
    return counter

if __name__ == '__main__':
    main(download_many, DEFAULT_CONCUR_REQ, MAX_CONCUR_REQ)